# From pandas to production

## Step 1: Let's train a model for twitter sentiment classification

### Prepare Dataset
Dataset source: https://www-cs.stanford.edu/people/alecmgo/papers/TwitterDistantSupervision09.pdf

In [ ]:
%%bash

if [ ! -f ./trainingandtestdata.zip ]; then
    wget -q http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
    unzip -n trainingandtestdata.zip
fi

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.pipeline import Pipeline

# Set the columns
columns = ['polarity', 'tweetid', 'date', 'query_name', 'user', 'text']
dftrain = pd.read_csv('training.1600000.processed.noemoticon.csv',
                      header = None,
                      encoding ='ISO-8859-1')
dftest = pd.read_csv('testdata.manual.2009.06.14.csv',
                     header = None,
                     encoding ='ISO-8859-1')
dftrain.columns = columns
dftest.columns = columns
dftrain.head()

### Model Training
Reference: https://docs.bentoml.org/en/latest/quickstart.html

In [ ]:
sentiment_lr = Pipeline([
                         ('count_vect', CountVectorizer(min_df = 100,
                                                        stop_words = 'english')), 
                         ('lr', LogisticRegression())])
sentiment_lr.fit(dftrain.text, dftrain.polarity)

In [ ]:
Xtest, ytest = dftest.text[dftest.polarity!=2], dftest.polarity[dftest.polarity!=2]
print(classification_report(ytest,sentiment_lr.predict(Xtest)))

## Step 2: Package the model into an API service

Now that we have a working machine learning model built with pandas, our goal is to share this model with our customers so they can make predictions on their data. 

Our customers do not have access to the jupyter notebook with our trained model, so we need to do some clever packaging to serve this model to our customers

A popular way to deploy our model to our customers is through a REST API, so that our customers can make a web request with their data, and get the model predictions as in the response.

### Create a Model Service using BentoML
BentoML is a python framework that makes it super easy to move your ML models as web APIs, so that your users can send data to a web URL and get the predictions of your model as the response!

In [ ]:
%%writefile sentiment_analysis_service.py

import bentoml

from bentoml.frameworks.sklearn import SklearnModelArtifact
from bentoml.service.artifacts.common import PickleArtifact
from bentoml.adapters import JsonInput

@bentoml.artifacts([PickleArtifact('model')])  # picke your trained model so that it can run on the server
@bentoml.env(pip_packages=["scikit-learn", "pandas"])  # specify the packages that your model depends on
class SKSentimentAnalysis(bentoml.BentoService):

    sentiment_names = {
        0: "very negative",
        1: "somewhat negative",
        2: "neutral",
        3: "somewhat positive",
        4: "very positive",
    }
    
    @bentoml.api(input=JsonInput())
    def predict(self, parsed_json):
        """
        Sentiment prediction API service
        
        Expected input format:
        ["Some text to predict the sentiment...", "some more text to predict sentiment"]

        Output format:
        {"sentiment_score": 4, "sentiment": "Very Positive", "tweet": "Tweet text to predict the sentiment..."}
        """
        texts = parsed_json
        predictions = self.artifacts.model.predict(texts)
        res = []
        for idx, pred in enumerate(predictions):
            res.append({
                "sentiment_score": pred, 
                "sentiment": self.sentiment_names[pred], 
                "text": texts[idx]
            })

        return res

## Step 3. Package the model service and the trained model into a Docker file for easy deployment

This step packages the model service with the trained model from step 1. After we package the API service with the model, we are ready to start serving some requests.

One common way of distributing this model API server for production deployment, is via Docker containers. And BentoML provides a convenient way to do that.

### 1) import the custom BentoService defined above
from sentiment_analysis_service import SKSentimentAnalysis

### 2) `pack` it with required artifacts, i.e. the trained model from step 1
bento_service = SKSentimentAnalysis()
bento_service.pack('model', sentiment_lr)

### 3) save your BentoSerivce to file archive
saved_path = bento_service.save()
!bentoml serve SKSentimentAnalysis:latest --port=5000

In [ ]:
def package_and_serve():
    # 1) import the custom BentoService defined above
    from sentiment_analysis_service import SKSentimentAnalysis

    # 2) `pack` it with required artifacts, i.e. the trained model from step 1
    bento_service = SKSentimentAnalysis()
    bento_service.pack('model', sentiment_lr)

    # 3) save your BentoSerivce to file archive
    saved_path = bento_service.save()

    # 4) Start a REST API model server with the BentoService saved above to serve the model
    !bentoml serve SKSentimentAnalysis:latest --port=5000
        
package_and_serve()

## Step 4: Error monitoring with Sentry

That's great, we have a Model API so we are done now, right? Not quite.

There are a few more steps before we can call our model API ready for production. For example, what happens if someone does not send the "tweet" field in the request?

Well, our API service expects that the user sends a "tweet" key in the JSON, so if they fail to send one, our server will error out.

As the author of the API service, you might want to know when the service encounters unexpected errors such as this

Sentry is great for this task!

In [ ]:
# Let's start by installing sentry
!pip3 install sentry-sdk

In [ ]:
%%writefile sentiment_analysis_service.py
# Now let's modify our service to use sentry

import bentoml

from bentoml.frameworks.sklearn import SklearnModelArtifact
from bentoml.service.artifacts.common import PickleArtifact
from bentoml.adapters import JsonInput

# Edit 1: Import sentry
import sentry_sdk
import logging
from sentry_sdk.integrations.logging import LoggingIntegration

# All of this is already happening by default!
sentry_logging = LoggingIntegration(
    level=logging.INFO,        # Capture info and above as breadcrumbs
    event_level=logging.ERROR  # Send errors as events
)
sentry_sdk.init(
    dsn="https://651a85506d1d4be3876a224d8a92eb2c@o1140265.ingest.sentry.io/6197278",
    integrations=[sentry_logging]
)


@bentoml.artifacts([PickleArtifact('model')])
@bentoml.env(infer_pip_packages=True)
class SKSentimentAnalysis(bentoml.BentoService):

    sentiment_names = {
        0: "very negative",
        1: "somewhat negative",
        2: "neutral",
        3: "somewhat positive",
        4: "very positive",
    }
    
    @bentoml.api(input=JsonInput())
    @bentoml.env(pip_packages=["scikit-learn", "pandas", "sentry-sdk==1.5.4"])
    def predict(self, parsed_json):
        """
        Sentiment prediction API service
        
        Expected input format:
        {"tweet": "Tweet text to predict the sentiment..."}

        Output format:
        {"sentiment_score": 4, "sentiment": "Very Positive", "tweet": "Tweet text to predict the sentiment..."}
        """
        # Edit 3: Update the code to capture exceptions to sentry
        try:
            texts = parsed_json
            predictions = self.artifacts.model.predict(texts)
            res = []
            for idx, pred in enumerate(predictions):
                # Edit 4: make a deliberate mistake
                1/ 0 # raise a ZeroDivisionError
                res.append({
                    "sentiment_score": pred, 
                    "sentiment": self.sentiment_names[pred], 
                    "text": texts[idx]
                })

            return res        
        except:
            sentry_sdk.capture_exception()
            return "error"

In [ ]:
package_and_serve()

## Step 5: Measure latency and throughput with Prometheus
Now that the API service is deployed, I would want to start measuring myAPI performance next. Latency and Throughput are two important API metrics that are useful to measure.

In [ ]:
%%writefile sentiment_analysis_service.py
# Now let's modify our service to use prometheus to measure latency

import bentoml

from bentoml.frameworks.sklearn import SklearnModelArtifact
from bentoml.service.artifacts.common import PickleArtifact
from bentoml.adapters import JsonInput

import sentry_sdk
import logging
from sentry_sdk.integrations.logging import LoggingIntegration

# All of this is already happening by default!
sentry_logging = LoggingIntegration(
    level=logging.INFO,        # Capture info and above as breadcrumbs
    event_level=logging.ERROR  # Send errors as events
)
sentry_sdk.init(
    dsn="https://your_dsn_goes_here@o1140265.ingest.sentry.io/934290",
    integrations=[sentry_logging]
)


# Edit 1: Import prometheus
from prometheus_client import Summary
REQUEST_TIME = Summary('request_processing_time', 'Time spend processing request')


@bentoml.artifacts([PickleArtifact('model')])
@bentoml.env(infer_pip_packages=True)
class SKSentimentAnalysis(bentoml.BentoService):

    sentiment_names = {
        0: "very negative",
        1: "somewhat negative",
        2: "neutral",
        3: "somewhat positive",
        4: "very positive",
    }
    
    # Edit 2: Monitor request time on the API
    @REQUEST_TIME.time()
    @bentoml.api(input=JsonInput())
    def predict(self, parsed_json):
        """
        Sentiment prediction API service
        
        Expected input format:
        {"tweet": "Tweet text to predict the sentiment..."}

        Output format:
        {"sentiment_score": 4, "sentiment": "Very Positive", "tweet": "Tweet text to predict the sentiment..."}
        """
        try:
            texts = parsed_json
            if len(texts) == 12:
                import time
                time.sleep(5)
            predictions = self.artifacts.model.predict(texts)
            res = []
            for idx, pred in enumerate(predictions):
                res.append({
                    "sentiment_score": pred, 
                    "sentiment": self.sentiment_names[pred], 
                    "text": texts[idx]
                })

            return res        
        except:
            sentry_sdk.capture_exception()
            return "error"

In [ ]:
package_and_serve()

#### Send prediction request to REST API server

Run the following command in terminal to make a HTTP request to the API server:
```bash
curl -i \
--header "Content-Type: application/json" \
--request POST \
--data '["some new text, sweet noodles", "happy time", "sad day"]' \
localhost:5000/predict
```

You can also view all availabl API endpoints at [localhost:5000](localhost:5000), or look at prometheus metrics at [localhost:5000/metrics](localhost:5000/metrics) in browser.

In [ ]:
%%writefile sentiment_analysis_service.py
# Now let's modify our service to use prometheus to measure custom metrics

import bentoml

from bentoml.frameworks.sklearn import SklearnModelArtifact
from bentoml.service.artifacts.common import PickleArtifact
from bentoml.adapters import JsonInput

import sentry_sdk
import logging
from sentry_sdk.integrations.logging import LoggingIntegration

# All of this is already happening by default!
sentry_logging = LoggingIntegration(
    level=logging.INFO,        # Capture info and above as breadcrumbs
    event_level=logging.ERROR  # Send errors as events
)
sentry_sdk.init(
    dsn="https://651a85506d1d4be3876a224d8a92eb2c@o1140265.ingest.sentry.io/6197278",
    integrations=[sentry_logging]
)

from prometheus_client import Summary
REQUEST_TIME = Summary('request_processing_time', 'Time spend processing request')
# Edit 1: Create a custom metric
REQUEST_TEXT_LEN = Summary('request_text_len', 'Length of texts array for inference')


@bentoml.artifacts([PickleArtifact('model')])
@bentoml.env(infer_pip_packages=True)
class SKSentimentAnalysis(bentoml.BentoService):

    sentiment_names = {
        0: "very negative",
        1: "somewhat negative",
        2: "neutral",
        3: "somewhat positive",
        4: "very positive",
    }
    
    @REQUEST_TIME.time()
    @bentoml.api(input=JsonInput())
    def predict(self, parsed_json):
        """
        Sentiment prediction API service
        
        Expected input format:
        {"tweet": "Tweet text to predict the sentiment..."}

        Output format:
        {"sentiment_score": 4, "sentiment": "Very Positive", "tweet": "Tweet text to predict the sentiment..."}
        """
        try:
            texts = parsed_json
            predictions = self.artifacts.model.predict(texts)
            res = []
            # Edit 2: Monitor request text lengt on the API
            REQUEST_TEXT_LEN.observe(len(texts))
            for idx, pred in enumerate(predictions):
                res.append({
                    "sentiment_score": pred, 
                    "sentiment": self.sentiment_names[pred], 
                    "text": texts[idx]
                })

            return res        
        except:
            sentry_sdk.capture_exception()
            return "error"

In [ ]:
# Repackage and serve
# 1) import the custom BentoService defined above
from sentiment_analysis_service import SKSentimentAnalysis

# 2) `pack` it with required artifacts, i.e. the trained model from step 1
bento_service = SKSentimentAnalysis()
bento_service.pack('model', sentiment_lr)

# 3) save your BentoSerivce to file archive
saved_path = bento_service.save()
!bentoml serve SKSentimentAnalysis:latest --port=5000